### Importation des modules qu'on a besoin

In [1]:
from pymongo import MongoClient
from typing import List, Dict
import polars as pl
import datetime as dt
import numpy as np

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.paris_listing
collection = db.listings

In [5]:
def mongodb_to_polars_records(data: List[Dict]) -> pl.DataFrame:
    """
    Convertit les données MongoDB en DataFrame Polars en utilisant from_records.
    
    Args:
        data: Liste de dictionnaires provenant de MongoDB
    
    Returns:
        pl.DataFrame: DataFrame Polars
    """
    # Supprimer le champ _id de MongoDB
    clean_data = []
    for record in data:
        # Créer une copie du dictionnaire sans _id
        clean_record = record.copy()
        if '_id' in clean_record:
            del clean_record['_id']
            
        # Convertir les bytes en str si nécessaire
        for key, value in clean_record.items():
            if isinstance(value, bytes):
                clean_record[key] = value.decode('utf-8')
                
        clean_data.append(clean_record)
    
    try:
        # Première tentative avec infer_schema_length par défaut
        return pl.from_records(clean_data)
    except Exception as e:
        print(f"Première tentative échouée, essai avec infer_schema_length plus grand: {str(e)}")
        try:
            # Deuxième tentative avec infer_schema_length plus grand
            return pl.from_records(clean_data, infer_schema_length=10000)
        except Exception as e:
            print(f"Deuxième tentative échouée, conversion en strings: {str(e)}")
            # Convertir toutes les valeurs en string
            str_data = [{k: str(v) if v is not None else "" for k, v in record.items()} 
                       for record in clean_data]
            return pl.from_records(str_data)

In [6]:
# Récupérer tous les documents
all_listings = list(collection.find())

# Convertir en DataFrame Polars
df = mongodb_to_polars_records(all_listings)

Première tentative échouée, essai avec infer_schema_length plus grand: could not append value: "" of type: str to the builder; make sure that all rows have the same schema or consider increasing `infer_schema_length`

it might also be that a value overflows the data-type's capacity


In [7]:
df.columns

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [8]:
# Calculer le taux de réservation moyen par mois par type de logement
df.with_columns(
    reservation_rate=(365 - pl.col("availability_365"))/ 365 * 100
) \
.group_by("property_type") \
.agg(avg_reservation_rate=pl.col("reservation_rate").mean().round(2)) \
.sort("avg_reservation_rate", descending=True)

property_type,avg_reservation_rate
str,f64
"""Entire bungalow""",100.0
"""Shared room in ice dome""",100.0
"""Dome""",100.0
"""Shared room in townhouse""",98.17
"""Cave""",97.26
…,…
"""Shipping container""",0.27
"""Tower""",0.27
"""Private room in villa""",0.27


In [98]:
# Calculer le pourcentage de reviews par type de chambre
df.select(["room_type", "number_of_reviews"]) \
.group_by("room_type") \
.agg(
    sum_reviews=pl.col("number_of_reviews").sum()
) \
.with_columns(
    pcg_reviews=(pl.col("sum_reviews") / pl.col("sum_reviews").sum() * 100).round(2)
)

room_type,sum_reviews,pcg_reviews
str,i64,f64
"""Entire home/apt""",1567261,82.19
"""Hotel room""",41909,2.2
"""Shared room""",18114,0.95
"""Private room""",279695,14.67


In [103]:
# Calculer la médiane des nombres d’avis pour tous les logements
df.select(pl.col("number_of_reviews").median()).item()

3.0

In [110]:
# Calculer la médiane des nombres d’avis par catégorie d’hôte
df.group_by("host_is_superhost").agg(
    pl.col("number_of_reviews").median()
).sort("number_of_reviews", descending=True)

host_is_superhost,number_of_reviews
str,f64
"""t""",24.0
"""""",12.5
"""f""",2.0


In [121]:
# Calculer la densité de logements par quartier de Paris
df.group_by("neighbourhood_cleansed").count().sort('count', descending=True)

/var/folders/b8/_p_1v95j6ql648xwdw382p9h0000gn/T/ipykernel_57724/4228890905.py:2: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  df.group_by("neighbourhood_cleansed").count().sort('count', descending=True)


neighbourhood_cleansed,count
str,u32
"""Buttes-Montmartre""",10555
"""Popincourt""",8430
"""Vaugirard""",7802
"""Batignolles-Monceau""",6857
"""Entrepôt""",6558
…,…
"""Élysée""",2898
"""Hôtel-de-Ville""",2821
"""Palais-Bourbon""",2740


In [128]:
# Identifier les quartiers avec le plus fort taux de réservation par mois
df.with_columns(
    reservation_rate=(365 - pl.col("availability_365"))/ 365 * 100
) \
.group_by("neighbourhood_cleansed") \
.agg(avg_neighbourhood_reservation_rate=pl.col("reservation_rate").mean().round(2)) \
.sort("avg_neighbourhood_reservation_rate", descending=True).limit(10)

neighbourhood_cleansed,avg_neighbourhood_reservation_rate
str,f64
"""Ménilmontant""",71.08
"""Buttes-Chaumont""",69.73
"""Buttes-Montmartre""",69.25
"""Entrepôt""",68.99
"""Popincourt""",68.87
"""Gobelins""",68.26
"""Reuilly""",67.74
"""Panthéon""",66.36
"""Vaugirard""",65.9
